In [1]:
from filtros import *
from functions import *
from fit_voc import *
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [8]:
def log_a(a,x):
    y = np.log(x)/np.log(a)
    return y

def nml_factors(df,eta_0,EOL_0,factors_column,cycles_column):
    # Ahora normalizamos los factores
    sr_numeric_0 = 100
    normal_factors = []
    eq_cycles = []

    for sr_k, factors in zip(df.SR,df[factors_column]):
        # obtenemos el sr de cada caso
        sr_range = [float(x) for x in sr_k.split("-")]
        sr_numeric = sr_range[0] - sr_range[1]
        # normlizamos el factor
        eta_k = factors*eta_0
        new_factor = (eta_k**(sr_numeric_0/sr_numeric))/eta_0
        eq_cycle = np.round(1/(log_a(EOL_0,eta_k**(sr_numeric_0/sr_numeric))),0)
        ## actualizamos los nuevos valores
        eq_cycles.append(eq_cycle)
        normal_factors.append(new_factor)
        # factors[0] = new_factor

    factors_column = "Factores_EQ"
    cycles_column = "Ciclos_EQ"
        
    df[factors_column] = normal_factors
    df[cycles_column] = eq_cycles
    return df

def adap_factors(df,new_cycles,old_cycles,new_EOL,factors_column,cycles_column):
    ## Adaptamos los factores a una nueva celda
    EOL_0 = 0.8
    new_eta0 = new_EOL**(1/new_cycles)

    # adaptamos para factores normalizados
    adapted_factors = []
    adapted_cycles = []

    for sr_k, factors in zip(df.SR,df[factors_column]):
        adap_factor = factors**(old_cycles/new_cycles)
        adap_etak = adap_factor*new_eta0
        adap_cycle = np.round(1/(log_a(new_EOL,adap_etak)),0)
        # actualizamos los nuevos valores
        adapted_cycles.append(adap_cycle)
        adapted_factors.append(adap_factor)
        # factors[0] = adap_factor
        
    factors_column = factors_column + "_adap"
    cycles_column = cycles_column + "_adap"

    df[factors_column] = adapted_factors
    df[cycles_column] = adapted_cycles
    return df

def unnml_factors(df,new_cycles,new_EOL,factors_column,cycles_column):
    new_eta0 = new_EOL**(1/new_cycles)
    sr_numeric_0 = 100
    unnoml_adapted_factors = []
    unnoml_adapted_cycles = []
    for sr_k, factor in zip(df.SR, df[factors_column]):
        # obtenemos el sr de cada caso
        sr_range = [float(x) for x in sr_k.split("-")]
        sr_numeric = sr_range[0] - sr_range[1]  # SR
        # normlizamos el factor
        eta_k = factor*new_eta0
        unnml_eta_k = eta_k**(sr_numeric/sr_numeric_0)
        unnml_factor = unnml_eta_k/new_eta0
        eq_cycle = 1/(log_a(new_EOL,unnml_eta_k))
        # actualizamos los nuevos valores
        unnoml_adapted_cycles.append(eq_cycle)
        unnoml_adapted_factors.append(unnml_factor)

    factors_column = factors_column + "_unnml"
    cycles_column = cycles_column + "_unnml"
    df[factors_column] = unnoml_adapted_factors
    df[cycles_column] = unnoml_adapted_cycles
    return df

def linear_factors(df,EOL_0,cycles_0,factors_column,cycles_column):
    lin_factors = []
    lin_cycles = []
    eta_0 = EOL_0**(1/cycles_0)
    sr_numeric_0 = 100
    for sr_k, factors in zip(df.SR, df[factors_column]):
        # obtenemos el sr de cada caso
        sr_range = [float(x) for x in sr_k.split("-")]
        sr_numeric = sr_range[0] - sr_range[1]
        lin_cycle = np.round(cycles_0*sr_numeric_0/sr_numeric,0)
        # normlizamos el factor
        eta_k = EOL_0**(1/lin_cycle)
        new_factor = eta_k/eta_0
        # eq_cycle = 1/(log_a(EOL_0,eta_k**(sr_numeric_0/sr_numeric)))
        ## actualizamos los nuevos valores
        lin_cycles.append(lin_cycle)
        lin_factors.append(new_factor)
        # factors[0] = new_factor

    factors_column = factors_column + "_linear"
    cycles_column = cycles_column + "_linear"
        
    df[factors_column] = lin_factors
    df[cycles_column] = lin_cycles
    return df

### Tenemos la tabla original de aramis

In [9]:
SR =         ["100-0    ",
              "100-25   ",
              "75-0     ",
              "100-50   ",
              "75-25    ",
              "50-0     ",
              "100-75   ",
              "75-50    ",
              "62.5-37.5",
              "50-25    ",
              "25-0     ",]
og08_factors = [1.00000000,
                1.00000266,
                1.00001860,
                0.99999203,
                1.00001521,
                1.00002874,
                1.00002146,
                1.00000881,
                1.00000620,
                1.00003347,
                1.00004184]
og_data = list(zip(SR,og08_factors))
og_headers = ["SR","Factors"]
df = pd.DataFrame(og_data,columns=og_headers)

# agregamos los subciclos que induce cada factor
cycles_0 = 4000
EOL_0 = 0.8
eta_0 = EOL_0**(1/cycles_0)
subc_k = []

for factor in df.Factors.values:
    eta_k = factor*eta_0
    sub_k = np.round(1/(log_a(EOL_0,eta_k)),0)
    subc_k.append(sub_k)
df["Subciclos"] = subc_k

df

,SR,Factors,Subciclos
0,100-0,1.000000,4000.0
1,100-25,1.000003,4200.0
2,75-0,1.000019,6001.0
3,100-50,0.999992,3500.0
4,75-25,1.000015,5499.0
5,50-0,1.000029,8250.0
6,100-75,1.000021,6501.0
7,75-50,1.000009,4750.0
8,62.5-37.5,1.000006,4500.0
9,50-25,1.000033,9999.0


In [14]:
new_cycles = 938
new_EOL = 0.8 # asumimos que el EOL es el mismo
df = nml_factors(df,eta_0,EOL_0,"Factors","Subciclos")
df = adap_factors(df,new_cycles,cycles_0,new_EOL,"Factores_EQ","Ciclos_EQ")
# df = unnml_factors(df,new_cycles,new_EOL,"Factors_nml_adap","Subciclos_nml_adap")


# df

,SR,Factors,Subciclos,Factores_EQ,Ciclos_EQ,Factores_EQ_adap,Ciclos_EQ_adap
0,100-0,1.000000,4000.0,1.000000,4000.0,1.000000,938.0
1,100-25,1.000003,4200.0,0.999985,3150.0,0.999936,739.0
2,75-0,1.000019,6001.0,1.000006,4501.0,1.000026,1055.0
3,100-50,0.999992,3500.0,0.999928,1750.0,0.999694,410.0
4,75-25,1.000015,5499.0,0.999975,2750.0,0.999892,645.0
5,50-0,1.000029,8250.0,1.000002,4125.0,1.000007,967.0
6,100-75,1.000021,6501.0,0.999918,1625.0,0.999652,381.0
7,75-50,1.000009,4750.0,0.999868,1188.0,0.999437,278.0
8,62.5-37.5,1.000006,4500.0,0.999857,1125.0,0.999392,264.0
9,50-25,1.000033,9999.0,0.999967,2500.0,0.999857,586.0


In [25]:
df = nml_factors(df,eta_0,EOL_0,"Factors","Subciclos")
df = adap_factors(df,new_cycles,cycles_0,new_EOL,"Factores_EQ","Ciclos_EQ")
df = linear_factors(df,EOL_0,938,"Factores_EQ_adap","Ciclos_EQ_adap")
df

,SR,Factors,Subciclos,Factores_EQ,Ciclos_EQ,Factores_EQ_adap,Ciclos_EQ_adap,freq_Ciclos_EQ,freq_Ciclos_EQ_adap,Factores_EQ_adap_linear,Ciclos_EQ_adap_linear,freq_Ciclos_EQ_adap_linear
0,100-0,1.000000,4000.0,1.000000,4000.0,1.000000,938.0,1.00,1.00,1.000000,938.0,1.00
1,100-25,1.000003,4200.0,0.999985,3150.0,0.999936,739.0,0.79,0.79,1.000060,1251.0,1.33
2,75-0,1.000019,6001.0,1.000006,4501.0,1.000026,1055.0,1.13,1.12,1.000060,1251.0,1.33
3,100-50,0.999992,3500.0,0.999928,1750.0,0.999694,410.0,0.44,0.44,1.000119,1876.0,2.00
4,75-25,1.000015,5499.0,0.999975,2750.0,0.999892,645.0,0.69,0.69,1.000119,1876.0,2.00
5,50-0,1.000029,8250.0,1.000002,4125.0,1.000007,967.0,1.03,1.03,1.000119,1876.0,2.00
6,100-75,1.000021,6501.0,0.999918,1625.0,0.999652,381.0,0.41,0.41,1.000178,3752.0,4.00
7,75-50,1.000009,4750.0,0.999868,1188.0,0.999437,278.0,0.30,0.30,1.000178,3752.0,4.00
8,62.5-37.5,1.000006,4500.0,0.999857,1125.0,0.999392,264.0,0.28,0.28,1.000178,3752.0,4.00
9,50-25,1.000033,9999.0,0.999967,2500.0,0.999857,586.0,0.62,0.62,1.000178,3752.0,4.00


In [15]:
df[["SR","Subciclos","Ciclos_EQ","Ciclos_EQ_adap"]]

,SR,Subciclos,Ciclos_EQ,Ciclos_EQ_adap
0,100-0,4000.0,4000.0,938.0
1,100-25,4200.0,3150.0,739.0
2,75-0,6001.0,4501.0,1055.0
3,100-50,3500.0,1750.0,410.0
4,75-25,5499.0,2750.0,645.0
5,50-0,8250.0,4125.0,967.0
6,100-75,6501.0,1625.0,381.0
7,75-50,4750.0,1188.0,278.0
8,62.5-37.5,4500.0,1125.0,264.0
9,50-25,9999.0,2500.0,586.0


In [20]:
frecuencies = []
cycle_0 = 4000
for cycle in zip(df["Ciclos_EQ"]):
    f = np.round(cycle[0]/cycle_0,2)
    frecuencies.append(f)
df["freq_Ciclos_EQ"] = frecuencies

frecuencies = []
cycle_0 = 938
for cycle in zip(df["Ciclos_EQ_adap"]):
    f = np.round(cycle[0]/cycle_0,2)
    frecuencies.append(f)
df["freq_Ciclos_EQ_adap"] = frecuencies

frecuencies = []
cycle_0 = 938
for cycle in zip(df["Ciclos_EQ_adap_linear"]):
    f = np.round(cycle[0]/cycle_0,2)
    frecuencies.append(f)
df["freq_Ciclos_EQ_adap_linear"] = frecuencies

df[["SR","Ciclos_EQ","Ciclos_EQ_adap","freq_Ciclos_EQ","freq_Ciclos_EQ_adap","freq_Ciclos_EQ_adap_linear"]]

,SR,Ciclos_EQ,Ciclos_EQ_adap,freq_Ciclos_EQ,freq_Ciclos_EQ_adap
0,100-0,4000.0,938.0,1.00,1.00
1,100-25,3150.0,739.0,0.79,0.79
2,75-0,4501.0,1055.0,1.13,1.12
3,100-50,1750.0,410.0,0.44,0.44
4,75-25,2750.0,645.0,0.69,0.69
5,50-0,4125.0,967.0,1.03,1.03
6,100-75,1625.0,381.0,0.41,0.41
7,75-50,1188.0,278.0,0.30,0.30
8,62.5-37.5,1125.0,264.0,0.28,0.28
9,50-25,2500.0,586.0,0.62,0.62


In [27]:
frecuencies = []
ciclos = []
cycle_0 = 938
for cycle in zip(df["Ciclos_EQ_adap_linear"]):
    f = 1
    frecuencies.append(f)
    ciclos.append(cycle_0)
df["freq_Ciclos_EQ_adap_linear"] = frecuencies
df["Ciclos_EQ_adap_linear"] = ciclos

df[["SR","Ciclos_EQ","Ciclos_EQ_adap","Ciclos_EQ_adap_linear","freq_Ciclos_EQ","freq_Ciclos_EQ_adap","freq_Ciclos_EQ_adap_linear"]]

,SR,Ciclos_EQ,Ciclos_EQ_adap,Ciclos_EQ_adap_linear,freq_Ciclos_EQ,freq_Ciclos_EQ_adap,freq_Ciclos_EQ_adap_linear
0,100-0,4000.0,938.0,938,1.00,1.00,1
1,100-25,3150.0,739.0,938,0.79,0.79,1
2,75-0,4501.0,1055.0,938,1.13,1.12,1
3,100-50,1750.0,410.0,938,0.44,0.44,1
4,75-25,2750.0,645.0,938,0.69,0.69,1
5,50-0,4125.0,967.0,938,1.03,1.03,1
6,100-75,1625.0,381.0,938,0.41,0.41,1
7,75-50,1188.0,278.0,938,0.30,0.30,1
8,62.5-37.5,1125.0,264.0,938,0.28,0.28,1
9,50-25,2500.0,586.0,938,0.62,0.62,1
